### [212\. Word Search II](https://leetcode.com/problems/word-search-ii/)

Difficulty: **Hard**


Given a 2D board and a list of words from the dictionary, find all words in the board.

Each word must be constructed from letters of sequentially adjacent cell, where "adjacent" cells are those horizontally or vertically neighboring. The same letter cell may not be used more than once in a word.

**Example:**

```
Input: 
board = [
  ['o','a','a','n'],
  ['e','t','a','e'],
  ['i','h','k','r'],
  ['i','f','l','v']
]
words = ["oath","pea","eat","rain"]

Output: ["eat","oath"]
```

**Note:**

1.  All inputs are consist of lowercase letters `a-z`.
2.  The values of `words` are distinct.

In [6]:
from collections import defaultdict

class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.isWord = False
        
class Trie:
    def __init__(self):
        self.root = TrieNode()
    
    def insert(self, word: str) -> None:
        node = self.root
        for ch in word:
            node = node.children[ch]
        node.isWord = True

In [19]:
# Trie (incorrect with third test case)
# https://leetcode.com/problems/word-search-ii/discuss/59790/Python-dfs-solution-(directly-use-Trie-implemented)
from typing import List,Generator

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # construct a Trie from words candidates
        trie = Trie()
        for w in words:
            trie.insert(w)
            
        def neighbors(i: int, j: int) -> Generator:
            for x, y in ((i+1,j),(i-1,j),(i,j+1),(i,j-1)):
                # move boundary check to dfs() to fix 1-length edge case
                yield(x,y)
        
        def dfs(i: int, j: int, node: TrieNode, stack: List[str], res: List[str]):
            if node.isWord:
                res.append(''.join(stack))
                node.isWord = False # mark this leaf node as visited
                return
            if not (0<=i<R and 0<=j<C): # boundary check
                return
            tmp = board[i][j]
            node = node.children.get(tmp)
            if not node:
                return
            board[i][j] = '#'
            for x, y in neighbors(i, j):
                dfs(x, y, node, stack + [tmp], res)
            board[i][j] = tmp
            
        if not board: return []
        R, C = len(board), len(board[0])
        res = []
        for i in range(R):
            for j in range(C):
                dfs(i, j, trie.root, [], res) # root is a placeholder
        return res

In [23]:
# faster solution
from typing import List,Generator

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # construct a Trie from words candidates
        trie = Trie()
        for w in words:
            trie.insert(w)
            
        def neighbors(i: int, j: int) -> Generator:
            for x, y in ((i+1,j),(i-1,j),(i,j+1),(i,j-1)):
                if 0<=x<R and 0<=y<C:
                    yield(x,y)
        
        def dfs(i: int, j: int, node: TrieNode, stack: List[str], res: List[str]):
            # board[i][j] is the key to access TrieNode (value) in the children dict
            if not node: # if key not found in parentNode.children, backtrack
                return
            if node.isWord:
                res.append(''.join(stack))
                node.isWord = False # mark this leaf node as visited
                # do not return here - edge case: [oat, oath] (some word is the prefix of the other word)
            tmp = board[i][j]
            board[i][j] = '#'
            for x, y in neighbors(i, j):
                dfs(x, y, node.children.get(board[x][y]), stack + [board[x][y]], res)
            board[i][j] = tmp
            
        if not board: return []
        R, C = len(board), len(board[0])
        res = []
        for i in range(R):
            for j in range(C):
                dfs(i, j, trie.root.children.get(board[i][j]), [board[i][j]], res) # root is a placeholder
        return res

In [24]:
Solution().findWords(board = [
  ['o','a','a','n'],
  ['e','t','a','e'],
  ['i','h','k','r'],
  ['i','f','l','v']
], words = ["oath","pea","eat","rain"])

['oath', 'eat']

In [25]:
Solution().findWords(board = [ ['a'] ], words = ["a"])

['a']

In [26]:
Solution().findWords(board = [
  ['o','a','a','n'],
  ['e','t','a','e'],
  ['i','h','k','r'],
  ['i','f','l','v']
], words = ["oath","pea","eat","rain","oat"])

['oat', 'oath', 'eat']